In [1]:
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt
import torch ## 저같은 경우엔 pytorch를 사용하여 학습하고 있어서 torch tensor로 저장하고자합니다
from tqdm import tqdm
import IPython.display

In [2]:
import numpy as np
from PIL import Image
import cv2
import os

def augment_image(image_path, output_dir):
    original_image = Image.open(image_path)
    base_name = os.path.splitext(os.path.basename(image_path))[0]

    for angle in [0, 45, 90, 135, 180, 225, 270, 315]:
        rotated_image = original_image.rotate(angle, expand=True, fillcolor='white')

        # Convert PIL Image to OpenCV format
        cv_image = cv2.cvtColor(np.array(rotated_image), cv2.COLOR_RGB2BGR)

        # Flip
        flipped_image = cv2.flip(cv_image, 1)  # 1 for horizontal flip
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_flipped.jpg"), flipped_image)

        # Save the original rotated image
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}.jpg"), cv_image)

def augment_all_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir, filename)
            augment_image(input_path, output_dir)

# 사용 예시
augment_all_images('cv_challenge/label_change/train', 'cv_challenge/data_argumentation_byteam_rotate_flipped')

In [3]:
%pwd

'C:\\Users\\master'

In [4]:
import csv
import os

def update_csv(original_csv_path, augmented_image_dir, output_csv_path):
    # 원본 CSV 파일 읽기
    with open(original_csv_path, 'r') as f:
        reader = csv.DictReader(f)
        original_data = list(reader)

    # 새로운 데이터를 저장할 리스트
    new_data = []

    # 증강된 이미지에 대한 데이터 추가
    for row in original_data:
        original_filename = row['ID']
        target = row['target']

        # 파일 이름에서 확장자 제거
        base_name = os.path.splitext(original_filename)[0]

        for angle in [0, 45, 90, 135, 180, 225, 270, 315]:
            # 회전된 이미지
            rotated_filename = f"{base_name}_rotated_{angle}.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, rotated_filename)):
                new_data.append({
                    'ID': rotated_filename,
                    'target': target
                })

            # Flipped 이미지
            flipped_filename = f"{base_name}_rotated_{angle}_flipped.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, flipped_filename)):
                new_data.append({
                    'ID': flipped_filename,
                    'target': target
                })
            
        # 원본이미지
        base_filename = f"{base_name}.jpg"
        if os.path.exists(os.path.join(augmented_image_dir, base_filename)):
            new_data.append({
                'ID': base_filename,
                'target': target
            })

    # 새로운 CSV 파일 작성
    with open(output_csv_path, 'w', newline='') as f:
        fieldnames = ['ID', 'target']
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        writer.writeheader()
        for row in new_data:
            writer.writerow(row)

    print(f"Updated CSV file with only augmented data has been saved to {output_csv_path}")

# 사용 예시
update_csv('cv_challenge/train.csv', 'cv_challenge/data_argumentation_byteam_rotate_flipped', 'cv_challenge/train_augmented_byteam_rf_and_original.csv')

Updated CSV file with only augmented data has been saved to cv_challenge/train_augmented_byteam_rf_and_original.csv


In [5]:
# 두 폴더(/home/data/train, /home/data/augmented_train)의 이미지 파일 합쳐서 하나의 새로운 폴더(/home/data/rotate_train)로 이동
import os
import shutil

def move_images(input_dir1, input_dir2, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir1):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir1, filename)
            output_path = os.path.join(output_dir, filename)
            shutil.copy(input_path, output_path)

    for filename in os.listdir(input_dir2):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir2, filename)
            output_path = os.path.join(output_dir, filename)
            shutil.copy(input_path, output_path)

move_images('cv_challenge/train', 'cv_challenge/data_argumentation_byteam_rotate_flipped', 'cv_challenge/augment_train_by_team_rf_and_original')

In [6]:
train_path = '/cv_challenge/data_argumentation_byteam_rotate_flipped'
cnt = 0
for root, dirs, files in os.walk(train_path):
    cnt += len(files)

print(f"Total number of files in {train_path}: {cnt}")

Total number of files in /cv_challenge/data_argumentation_byteam_rotate_flipped: 0


In [7]:
import pandas as pd

data = 'cv_challenge/train_augmented_byteam_rf_and_original.csv'
data = pd.read_csv(data)
data.head(20)

,ID,target
0,002f99746285dfdd_rotated_0.jpg,16
1,002f99746285dfdd_rotated_0_flipped.jpg,16
2,002f99746285dfdd_rotated_45.jpg,16
3,002f99746285dfdd_rotated_45_flipped.jpg,16
4,002f99746285dfdd_rotated_90.jpg,16
5,002f99746285dfdd_rotated_90_flipped.jpg,16
6,002f99746285dfdd_rotated_135.jpg,16
7,002f99746285dfdd_rotated_135_flipped.jpg,16
8,002f99746285dfdd_rotated_180.jpg,16
9,002f99746285dfdd_rotated_180_flipped.jpg,16
